# Challenge 05 - Red Teaming

## 1. Overview 

## 2. Setting Up Environment 

In [ ]:
# Azure imports
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# OpenAI imports
from openai import AzureOpenAI
import asyncio
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

## 2. Creating Red Teaming Agent

In [ ]:
## Using Azure AI Foundry project from environment variable
azure_ai_project = os.getenv("AZURE_AI_PROJECT")

# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential() # required
)

## 3. OpenAI Client and callback

In [ ]:
# Create Azure OpenAI client using managed identity (recommended approach)
def get_azure_ad_token():
    """Function to get Azure AD token for OpenAI authentication"""
    credential = DefaultAzureCredential()
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    return token.token

azure_openai_client = AzureOpenAI(
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_ad_token_provider=get_azure_ad_token
)

# Target callback function that uses your Azure OpenAI deployment
def azure_openai_callback(query: str) -> str:
    """
    This function represents your AI system that will be red-teamed.
    It sends the query to your Azure OpenAI deployment and returns the response.
    """
    try:
        response = azure_openai_client.chat.completions.create(
            model=os.getenv("CHAT_MODEL_NAME2"),  # This should match your deployment name
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": query}
            ],
            max_tokens=500,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

## 4. Running the Red Team Agent

In [ ]:
async def main():
    # The scan method expects a target function, not a configuration object
    red_team_result = await red_team_agent.scan(
        target=azure_openai_callback,
        risk_categories=[ # optional, defaults to all four risk categories
            RiskCategory.Violence,
            RiskCategory.HateUnfairness,
            RiskCategory.Sexual,
            RiskCategory.SelfHarm
        ],
        num_objectives=5  # Number of conversation turns per attack
    )
    print("Red team scan completed!")
    print(red_team_result)

if __name__ == "__main__":
    asyncio.run(main())